In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
from src.training import *
from src.pipeline.loaders import collate_numpy_matrices_without_conditions
from src.pipeline.datasets import RotationalDataset, prepare_and_save_seed_dataset, MatrixTupleDataset
import warnings
warnings.filterwarnings("ignore", message="Unknown entity 'ee-infinity-loader'")

filename = 'data/processed/paths.npz'
dims = (20,20)

# if not os.path.exists(filename):
#     prepare_and_save_seed_dataset(filename, dims, repr_version=5, seed_paths=10)

matrix_dataset = MatrixTupleDataset(filename)
rotational_datalist = RotationalDataset(matrix_dataset)

dataloader = DataLoader(
    rotational_datalist,
    batch_size=256, 
    collate_fn=collate_numpy_matrices,
    num_workers=0,
    pin_memory=True
)
train_dataloader, test_dataloader = split_dataloader(dataloader, val_split=0.2)
train_dataloader, val_dataloader = split_dataloader(train_dataloader, val_split=0.2)

input_size = dims[0]

print(f"Dataset size is: {len(rotational_datalist)}")

Dataset size is: 22900


In [ ]:
from src.model import BinaryMatrixTransformCNN, AttentiveBinaryMatrixTransformCNN, FactorioCNN_PixelOutput
from src.training import train_model
import itertools

# model1 = BinaryMatrixTransformCNN(matrix_size=input_size)
# model2 = AttentiveBinaryMatrixTransformCNN(matrix_size=input_size)

models = []
input_size = dims[0]

# underscore as to not shadow str
for str_, bias in itertools.product([0.5, 1, 1.75], [-0.5, 0, 0.5]):
    moodel = FactorioCNN_PixelOutput(4, 4, 5, 3, 3, 2, 32, 21, 
                                    presence_gate_strength=str_,
                                    presence_gate_bias=bias)
    moodel.filename += f"_pgst{str_}_bias{bias}"
    train_model(moodel, train_loader=train_dataloader,
                val_loader=val_dataloader,
                integrity_weight=0.0, num_epochs=100, device='cuda')
    del(moodel)  # great farm in the sky

: 